# NHL Data Cleaning (amateur workflow)

This notebook shows a simple, structured data cleaning flow.
Focus: checks, basic cleaning, filling gaps, simple categorization, and a quick chart.


In [ ]:
# SECTION: Imports
# pandas for tables, numpy for numbers, matplotlib for charts, re for text parsing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re


In [ ]:
# SECTION: Load data and quick look
df = pd.read_csv('NHLScraper/nhl.csv')
df.head()
df.shape
df.info()
df.describe(include='all')


In [ ]:
# SECTION: Basic checks
# Missing values per column
missing = df.isna().sum().sort_values(ascending=False)
missing

# Duplicate rows check
duplicates = df.duplicated().sum()
duplicates

# Simple type checks
type_checks = {
    'Year_is_numeric': pd.api.types.is_numeric_dtype(df['Year']),
    'Wins_is_numeric': pd.api.types.is_numeric_dtype(df['Wins']),
    'Losses_is_numeric': pd.api.typesnumis_numeric_dtype(df['Losses']),
}
type_checks


In [ ]:
# SECTION: Cleaning and filling gaps
# 1) Text cleanup
df['Team_Name'] = df['Team_Name'].astype(str).str.strip()

# 2) Year to int (safe conversion)
df['Year'] = pd.to_datetime(df['Year'], errors='coerce').dt.year

# 3) Make numeric columns really numeric
num_cols = ['Wins', 'Losses', 'Overtime_Losses', 'Goals_For', 'Goals_Against', 'Goal_Differential', 'Win_Percentage']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4) Win percentage: if in 0-1 scale, convert to 0-100
if df['Win_Percentage'].max() <= 1:
    df['Win_Percentage'] = (df['Win_Percentage'] * 100).round(2)

# 5) Fill gaps (simple, amateur-friendly)
df['Overtime_Losses'] = df['Overtime_Losses'].fillna(0).astype(int)
df['Team_Name'] = df['Team_Name'].replace({'nan': 'Unknown'})
for col in num_cols:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())


In [ ]:
# SECTION: Simple features and categorization
# Games played and per-game stats
df['Games_played'] = df['Wins'] + df['Losses'] + df['Overtime_Losses']
df.loc[df['Games_played'] == 0, 'Games_played'] = pd.NA
df['GF_per_Game'] = (df['Goals_For'] / df['Games_played']).round(2)
df['GA_per_Game'] = (df['Goals_Against'] / df['Games_played']).round(2)
df['GD_per_Game'] = (df['Goal_Differential'] / df['Games_played']).round(2)

# Simple z-score comparison within each year
df['GF_z_in_year'] = df.groupby('Year')['GF_per_Game'].transform(lambda s: (s - s.mean()) / s.std()).round(2)
df['GA_z_in_year'] = df.groupby('Year')['GA_per_Game'].transform(lambda s: (s - s.mean()) / s.std()).round(2)
df['Performance_Index'] = (df['GF_z_in_year'] - df['GA_z_in_year']).round(2)

# Team initials from name
def initials(name: str) -> str:
    words = re.findall(r'[A-Za-z]+', str(name))
    return ''.join(w[0].upper() for w in words if w)

df['Team_Initials'] = df['Team_Name'].apply(initials)

# Simple era categorization
df['Era'] = np.where(df['Year'] < 2000, '1990s', '2000s')


In [ ]:
# SECTION: Validation after cleaning
checks = {
    'no_missing_team': df['Team_Name'].notna().all(),
    'year_range_ok': df['Year'].between(1990, 2011).all(),
    'wins_non_negative': (df['Wins'] >= 0).all(),
    'losses_non_negative': (df['Losses'] >= 0).all(),
    'goals_non_negative': (df['Goals_For'] >= 0).all() and (df['Goals_Against'] >= 0).all(),
    'team_year_unique': (df.duplicated(['Team_Name', 'Year']).sum() == 0),
    'goal_diff_consistent': (df['Goal_Differential'] == (df['Goals_For'] - df['Goals_Against'])).all(),
}
checks


In [ ]:
# SECTION: Save cleaned data
df.to_csv('nhl_cleaned.csv', index=False)


## Quick chart
Average wins by team initials (simple view).


In [ ]:
# SECTION: Simple visualization
top_teams = (
    df.groupby('Team_Initials')['Wins']
      .mean()
      .sort_values(ascending=False)
      .head(10)
      .sort_values()
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(top_teams.index, top_teams.values)
ax.set_title('Top Teams by Average Wins')
ax.set_xlabel('Average Wins')
ax.set_ylabel('Teams')
ax.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
